In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
import os
import pandas as pd
current_dir = os.getcwd()

In [4]:

train_path = f"{current_dir}/train.csv"
test_path = f"{current_dir}/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

cols = ['ImageFilePath', 'Genus']
train_df = train_df[cols].dropna()
test_df = test_df[cols].dropna()

In [5]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")

tokenizer = open_clip.get_tokenizer("ViT-B-32")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False


In [6]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")

    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()


In [7]:

X_train = np.vstack([extract_features(img) for img in tqdm(train_df["ImageFilePath"], desc="Processing Training Images...")])

X_test = np.vstack([extract_features(img) for img in tqdm(test_df["ImageFilePath"], desc="Processing Testing Images...")])

le = LabelEncoder()

all_labels = pd.concat([train_df["Genus"], test_df["Genus"]]).unique()

le.fit(all_labels)

y_train = le.transform(train_df["Genus"])

y_test = le.transform(test_df["Genus"])

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)



Processing Training Images...:   0%|          | 0/35951 [00:00<?, ?it/s]

Processing Testing Images...:   0%|          | 0/9439 [00:00<?, ?it/s]

In [8]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T



Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 28.62% | Prec: 36.75% | Rec: 28.62% | F1: 29.93% | Bal Acc: 28.95% | MCC: 0.2159
LogisticRegression        | Acc: 20.73% | Prec: 41.73% | Rec: 20.73% | F1: 24.23% | Bal Acc: 17.08% | MCC: 0.1718
NearestNeighbor           | Acc: 24.68% | Prec: 47.59% | Rec: 24.68% | F1: 23.32% | Bal Acc: 31.37% | MCC: 0.1495
MLP_Baseline              | Acc: 30.76% | Prec: 47.96% | Rec: 30.76% | F1: 28.54% | Bal Acc: 31.19% | MCC: 0.2161


In [9]:

test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

test_df.head(2)



,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Beetle...,Chlaenius,Bembidion,Carabus,Calathus,Pterostichus
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Beetle...,Chlaenius,Calathus,Pterostichus,Amara,Pterostichus


In [10]:

# basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_10_per_species/'


In [11]:

test_df.to_csv(f"{current_dir}/2.CLIP.csv", index=False)

metrics_df.to_csv(f"{current_dir}/2.CLIP-metrics.csv", index=False)

